In [ ]:
# 엑셀파일이 'phenotype mappingv3.xlsx', '1_검증.csv' 이렇게 2개가 있어
#  'phenotype mappingv3.xlsx' 파일은 UMLSID, Group, GroupID 이렇게 3개의 컬럼이 존재하고
# '1_검증.csv' 파일은 Phenotype, Network_score, z-score, P-value, UMLSID 이렇게 5개의 컬럼이 존재해
# 여기서 '1_검증.csv' 파일 내에 있는 UMLSID의 데이터와  'phenotype mappingv3.xlsx' 파일 내에 있는 UMLSID의 데이터가 일치하면 Group, GroupID 값을 가져와서 똑같이 써주고, 아니면 None 값을 써주고 싶어
# 그래서 최종적으로 '1_검증.csv' 파일내에 Phenotype, Network_score, z-score, P-value, UMLSID, Group, GroupID 이렇게 7개의 컬럼이 존재하게 만들고 싶어
# 이렇게 코드 짜줘

import numpy as np
import pandas as pd
import configparser
import psycopg2
import os
import ast
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
from collections import defaultdict
from statistics import harmonic_mean
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# phenotype_mapping_file='/data/home/sss2061/식약처보고서/서울과기대/phenotype mappingv3.xlsx'

# '1_검증.csv' 파일을 읽어옵니다.

number=2 #수정 필요
validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number)) #수정 필요

# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('phenotype mappingv3.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
merged_df = validation_df.merge(mapping_df, on='UMLSID', how='left')

# Group 및 GroupID가 없는 경우 None으로 채웁니다.
merged_df['Group'].fillna('None', inplace=True)
merged_df['GroupID'].fillna('None', inplace=True)

# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = merged_df['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
merged_df['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

merged_df['P-value'] = merged_df['P-value'].apply(adjust_pvalue)

cols_to_drop = [col for col in merged_df.columns if col.startswith("Unnamed")]
merged_df = merged_df.drop(columns=cols_to_drop)

hpo_df=pd.read_csv("Data/[original] HPO phenotype data.csv")

merged_df=pd.merge(merged_df, hpo_df, left_on='Phenotype', right_on='phenotype name', how='left')

final_df=merged_df[['Phenotype','Network_score','z-score','P-value','UMLSID','Group','GroupID','scaling_score', 'hpo_id']]

# 수정된 DataFrame을 새로운 엑셀 파일로 저장합니다.
final_df.to_csv('result/{}차/{}_검증_with_scaling.csv'.format(number,number), index=False)  # 저장할 파일명을 지정해주세요.

In [ ]:
# 백업본

# 엑셀파일이 'phenotype mappingv3.xlsx', '1_검증.csv' 이렇게 2개가 있어
#  'phenotype mappingv3.xlsx' 파일은 UMLSID, Group, GroupID 이렇게 3개의 컬럼이 존재하고
# '1_검증.csv' 파일은 Phenotype, Network_score, z-score, P-value, UMLSID 이렇게 5개의 컬럼이 존재해
# 여기서 '1_검증.csv' 파일 내에 있는 UMLSID의 데이터와  'phenotype mappingv3.xlsx' 파일 내에 있는 UMLSID의 데이터가 일치하면 Group, GroupID 값을 가져와서 똑같이 써주고, 아니면 None 값을 써주고 싶어
# 그래서 최종적으로 '1_검증.csv' 파일내에 Phenotype, Network_score, z-score, P-value, UMLSID, Group, GroupID 이렇게 7개의 컬럼이 존재하게 만들고 싶어
# 이렇게 코드 짜줘

import numpy as np
import pandas as pd
import configparser
import psycopg2
import os
import ast
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
from collections import defaultdict
from statistics import harmonic_mean

# phenotype_mapping_file='/data/home/sss2061/식약처보고서/서울과기대/phenotype mappingv3.xlsx'

# '1_검증.csv' 파일을 읽어옵니다.

number=2
validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number))

# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('phenotype mappingv3.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
merged_df = validation_df.merge(mapping_df, on='UMLSID', how='left')

# Group 및 GroupID가 없는 경우 None으로 채웁니다.
merged_df['Group'].fillna('None', inplace=True)
merged_df['GroupID'].fillna('None', inplace=True)

# 결과를 'merged_1_검증.csv' 파일로 저장합니다.
merged_df.to_csv('merged_14_검증.csv', index=False)

import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 'merged_1_검증.xlsx' 파일을 읽어옵니다.
df = pd.read_csv('merged_14_검증.csv')  # 파일 경로를 확인해주세요.

# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = df['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
df['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

df['P-value'] = df['P-value'].apply(adjust_pvalue)

# "Unnamed"로 시작하는 컬럼을 찾아 삭제합니다.
cols_to_drop = [col for col in df.columns if col.startswith("Unnamed")]
df = df.drop(columns=cols_to_drop)

# 수정된 DataFrame을 새로운 엑셀 파일로 저장합니다.
df.to_csv('merged_14_검증_with_scaling.csv', index=False)  # 저장할 파일명을 지정해주세요.

In [ ]:
# number=1
# text=["adenine", "canavanin", "malate", "isoleucine", "phenylalanine", "tyrosine", "isocitrate", "glutamine", "tryptophan","L-malate", 
#       "theogallin", "trans-aconitate", "cis-aconitate", "threonate", "benzoic acid", "choline", "citric acid", "coumarin", "phloroglucinol", "gallic acid", 
#       "citrullin", "pipecolic acid", "vitamin B", "pantothenic acid", "DL-valine", "vanillin", "theophylline", "caffeine", "protocatechuic aldehyde", "adenosine", 
#       "EGCG", "gallocatechin", "N-acetylglutamate", "loliolide", "procyanidin B2", "3H-proline", "dihydromyricetin", "beta-alanine betaine", "epicatechin gallate (ECG", "5'-methylthioadenosine", 
#       "epiafzelechin", "ferulic acid", "p-coumaric acid", "caffeic acid", "chlorogenic acid", "quercetin", "alpha-isopropylmalate", "neochlorogenic acid", "rutin", "hyperoside", 
#       "4-p-coumaroylquinic acid", "astragalin", "quercetin 7-O-glucoside", "p-coumaroylquinic acid", "procyanidin B1"]

# number=2
# text=["gamma-aminobutyric acid (GABA", "canavanin", "sucrose", "folate", "guanosine", "vanillic acid", "ononin", 
#       "formononetin", "calycosin", "sissotrin", "afrormosin", "pratensein"]

# number=3
# text=["protocatechuic acid", "gamma-aminobutyric acid (GABA", "choline", "citric acid", "malate", "asparagine", 
#       "quinic acid", "maltol", "nodakenetin", "nodakenin", "columbianetin", "decursin", "ferulic acid", "caffeic acid",
#       "aegelinol", "cis-chlorogenic acid", "chlorogenic acid", "esculetin"]

# number=4
# text=["piperine", "piperanine", "piperlonguminine", "retrofractamide B", "dehydropipernonaline", "Methyl piperate", "pipernonaline", "Retrofractamide A", "retrofractamide C"]

# number=5
# text=["apocynin", "resacetophenone", "4-hydroxyacetophenone", "2,5-dihydroxyacetophenone", "physcion"]

# number=6
# text=["protocatechuic acid", "p-hydroxybenzaldehyde", "choline", "gluconate", "myo-inositol", "thymine", "uracil", "vanillin", "vanillic acid", "protocatechuic aldehyde", "trans-aconitate", "ferulic acid", "caffeic acid", "chlorogenic acid", "linoleic acid", "neochlorogenic acid", "xanthatin", "cynarin", "cynarine"]

# number=7
# text=["atractylenolide III", "vanillic acid", "esculetin", "elemicin", "adenosine"]


In [4]:
#결과 후처리

# rwr 조화평균 완료

# herb 최종
from tqdm import tqdm_notebook
import numpy as np
import pickle
import pandas as pd
import configparser
import psycopg2
import os
import ast
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
import copy
from collections import defaultdict
from statistics import harmonic_mean

start = time.time()

def calculate_mean(numbers):
    return sum(numbers) / len(numbers)

def calculate_standard_deviation(numbers):
    return statistics.stdev(numbers)

def combine_lists(full_list):
    combined = defaultdict(list)
    
    # 모든 리스트에 대한 처리
    for sublist in full_list:
        for item in sublist:
            entrez_id, rwr_score = item
            combined[entrez_id].append(rwr_score)
    
    # 조화평균 및 결과 리스트 만들기
    result = []
    for entrez_id, scores in combined.items():
        if len(scores) == 1:
            result.append([entrez_id, scores[0]])  # 값이 하나면 그대로 추가
        else:
            # 조화평균 계산
            harmonic_mean_score = harmonic_mean(scores)
            result.append([entrez_id, harmonic_mean_score])
    
    return result

def compound_target(compoundid):

    conn_cur.execute(f"with tmp1 as (select cg.geneid from compound_gene cg where (cg.compid = '{compoundid}' and cg.relationsign in ('activate','inhibit')))" 
                    f"select g.entrezid from gene g,tmp1 where g.geneid=tmp1.geneid;")
    compound_gene1=conn_cur.fetchall()

    compound_gene=set(compound_gene1)
    compound_gene=list(compound_gene)
    
    # Compound-target gene List 생성
    target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            target_gene_list.append(interaction_gene)        
        else:
            continue

    # restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in target_gene_list:
        node_ = node_index.get(entrez) 
        target_node.append(node_)

    conn_cur.execute(f"with tmp2 as (select cg.geneid from compound_gene cg where cg.compid = '{compoundid}' and relationsign='associate')" 
                    f"select g.entrezid from gene g,tmp2 where g.geneid=tmp2.geneid;")

    compound_gene2=conn_cur.fetchall()

    compound_gene=set(compound_gene2)
    compound_gene=list(compound_gene)

    indirected_target_gene_list = [] 
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            indirected_target_gene_list.append(interaction_gene)        
        else:
            continue      

    num1=len(target_gene_list) #target gene list 갯수
    directed_target_gene_list.append(num1)
    num2=len(indirected_target_gene_list) #indirected target gene list 갯수 
    indirected_target_gene_list_count.append(num2)
    
    #indirect nodes 정의
    indirect_nodes= []
    for entrez in indirected_target_gene_list:
        node_ = node_index.get(entrez) 
        indirect_nodes.append(node_)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 

    # RWR 실행
    rwr = RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    
    var_name1 = f'{compoundid}_num1'
    var_name2 = f'{compoundid}_num2'

    # 전역 변수 설정
    globals()[var_name1] = num1
    globals()[var_name2] = num2
    
    return rwr_mapping_entrez

def rwr_compound_target(count,entrez_list):
    
    num1=directed_target_gene_list[count]
    num2=indirected_target_gene_list_count[count]
    
    random_interaction_gene1 = random.sample(entrez_list, num1)
    remaining_entrez_list = [gene for gene in entrez_list if gene not in random_interaction_gene1]
    random_interaction_gene2 = random.sample(remaining_entrez_list, num2)
    # restart node에 적용할 target gene List 생성
    target_node = [] 
    for entrez in random_interaction_gene1:
        node_ = node_index.get(entrez)
        target_node.append(node_)

    #indirect nodes 정의
    indirect_nodes= []
    for entrez in random_interaction_gene2:
        node_ = node_index.get(entrez)
        indirect_nodes.append(node_)

    # PPI network RWR을 위한 initialize
    initialized = RWR_initializing(G, seed_nodes=target_node,indirect_target_nodes=indirect_nodes, is_weighted=False) 
    
    # RWR 실행
    rwr=RWR(initialized=initialized, prob=0.2, max_iter=100, tol=1.0e-6)
    rwr_mapping_entrez = [[entrez_list[i], rwr[node_index.get(entrez_list[i])]] for i in range(len(entrez_list))]
    return rwr_mapping_entrez

def create_ppi_network(network_data):
    network_data = network_data.astype({'Entrez Gene Interactor A': str,
                                        'Entrez Gene Interactor B': str})

    # symbol list를 생성하고 unique 값만 가지도록 생성
    symbolA = network_data.loc[:, 'Entrez Gene Interactor A'].to_list()
    symbolB = network_data.loc[:, 'Entrez Gene Interactor B'].to_list()

    symbol_list = symbolA + symbolB 
    symbol_list = set(symbol_list) 
    symbol_list = list(symbol_list)

    # 모든 symbol에 index를 딕셔너리에서 부여
    node_index = {}
    for i in range(len(symbol_list)):
        node_index[symbol_list[i]] = i

    # index로 node_list 생성
    node_list = node_index.values()
    
    # index로 edge_list 생성
    edge_list = network_data[['Entrez Gene Interactor A', 'Entrez Gene Interactor B']].values.tolist()  #
    for i in range(len(edge_list)):
        edge_list[i][0] = node_index.get(edge_list[i][0])
        edge_list[i][1] = node_index.get(edge_list[i][1])
    
    # 무방향 그래프 오브젝트 생성
    G = nx.Graph()

    # 그래프에 노드 추가
    G.add_nodes_from(node_list)

    # 그래프에 엣지 추가
    G.add_edges_from(edge_list)

    print(f'Number of created nodes: {G.number_of_nodes()}')
    print(f'Number of created edges: {G.number_of_edges()}')

    return G, node_index

def RWR_initializing(G, seed_nodes, indirect_target_nodes, is_weighted=False):
    norm_A = np.zeros(shape=(len(G), len(G)))
    if is_weighted:  
        for i, neighbor_dict in G.adjacency():
            for j, v in neighbor_dict.items():
                norm_A[i][j] = v.get("weight", 1 / len(neighbor_dict))
    else:  
        for i, neighbor_dict in G.adjacency():  
            for j, v in neighbor_dict.items():  
                norm_A[i][j] = 1 / len(neighbor_dict)
    
    # seed nodes와 indirect target nodes를 모두 포함한 딕셔너리 생성
    personalization = {node: 1 for node in seed_nodes}
    for node in indirect_target_nodes:
        personalization[node] = 0.3  
        
    r_0 = np.array([personalization.get(n, 0) for n in range(len(G))]) 
    r_c = np.repeat(1 / len(G), len(G)) 
    return {"norm_A": norm_A, "r_0": r_0, "r_c": r_c, "N": len(G)}

def RWR(initialized, prob=0.85, max_iter=100, tol=1.0e-6):
    if initialized is None:
        raise ValueError('initialized information must be required')

    norm_A = initialized['norm_A']  # 인접행렬 A (정규화된)
    r_0 = initialized['r_0']        # 시작 노드
    r_c = initialized['r_c']        # 가중치 행렬
    N = initialized['N']            # 노드의 개수
    for iteration in range(max_iter):
        r_prev = r_c
        r_c = prob * r_c @ norm_A + (1 - prob) * r_0  # RWR algorithm
        err = np.absolute(r_c - r_prev).sum()
        if err < N * tol:
#             print(f'RWR iteration = {iteration + 1}')
#             print('Converged')
            return r_c
#         else:
#             print(f'RWR iteration = {iteration + 1}, Iteration until convergence ...')
#             print(f'{err} -> {N * tol}')
    return "NotConverged"

In [17]:
#결과 후처리

from tqdm import tqdm_notebook
import numpy as np
import pickle
import pandas as pd
import configparser
import psycopg2
import os
import ast
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
import copy
from collections import defaultdict
from statistics import harmonic_mean
import pickle

number=7

with open("result/{}차/output_dict.pickle".format(number),"rb") as fi:
    output_dict = pickle.load(fi)
    
# validation_df = pd.read_csv('result/{}차/{}_검증.csv'.format(number,number))
inferred_phenotype2 = pd.read_csv('result/{}차/{}_network_score.csv'.format(number,number))

phenotype_rwr_score_dict = pd.Series(inferred_phenotype2.Network_score.values, index=inferred_phenotype2.Phenotype).to_dict()

temp=list(phenotype_rwr_score_dict.keys())
for i in temp:
    if np.isnan(phenotype_rwr_score_dict[i])==True:
        phenotype_rwr_score_dict.pop(i)

Phenotype_score_Phenotype=list(phenotype_rwr_score_dict.keys())
Phenotype_score_Score=list(phenotype_rwr_score_dict.values())

temp2=list(output_dict.keys())
for i in temp2:
    if np.isnan(output_dict[i][0])==True:
        output_dict.pop(i)
        
P_value_Phenotype=list(output_dict.keys())
P_value_Score=list(output_dict.values())


In [18]:
count=0
z_score_list=[]
outlier_list=[] #제거가능
P_value_list=[]
rank_number=0
result_phenotype=[]
result_phenotype_score=[]

for i in range(len(P_value_Score)):
    Numberlist=P_value_Score[i].copy()
    tmp1=P_value_Phenotype[i]
    tmp2=Phenotype_score_Phenotype.index(tmp1)
    tmp3=Phenotype_score_Score[tmp2]
    Numberlist.append(tmp3)
    result_phenotype.append(tmp1)
    result_phenotype_score.append(tmp3)
    
    # 평균 계산
    mean = calculate_mean(Numberlist)
    # 표준편차 계산
    std_deviation = calculate_standard_deviation(Numberlist)

    z_score=(Phenotype_score_Score[i]-mean)/std_deviation
    z_score_list.append(z_score)
    if z_score >=-2.58 and z_score<=2.58: #제거가능
        outlier=1
        count=count+1
    else:
        outlier=0 
    outlier_list.append(outlier) #제거가능

    Numberlist.sort(reverse=True)

    rank=Numberlist.index(tmp3) #순위찾기
    if rank<5:
        rank_number=rank_number+1
    P_value_list.append(rank)
    print("phe",tmp1)
    print("z",z_score)
    print("p",rank)

# postgresql DB 연동
config = configparser.ConfigParser()
config.read('db_config.ini')
conn=psycopg2.connect(host="168.131.30.66", dbname="coconut",user="dbuser",password="jnudl1") #데이터이름변경
conn_cur = conn.cursor()
#------------------------------------
phenotype_umlsid = []
# phenotype_list를 순회하면서 각 이름에 대한 UMLS ID를 가져옴
for phen_name in Phenotype_score_Phenotype:
    # 이름에 단일 따옴표가 포함된 경우 이스케이프
#     escaped_name = phen_name.replace("'", "''")
    
#     conn_cur.execute(f"SELECT umlsid FROM phenotype WHERE name = '{escaped_name}'")
    conn_cur.execute("SELECT umlsid FROM phenotype WHERE name = %s",(phen_name,))
    umlsid_result = conn_cur.fetchone()
    
    # UMLS ID가 존재하면 리스트에 추가
    if umlsid_result:
        phenotype_umlsid.append(umlsid_result[0])
    else:
        # UMLS ID가 없는 경우에 대한 처리 (예: None 또는 다른 기본값)
        phenotype_umlsid.append(None)

# 결과를 DataFrame으로 만들어 저장
phenotype_umlsid_df = pd.DataFrame({'Name': Phenotype_score_Phenotype, 'UMLSID': phenotype_umlsid})

#------------------------

Phenotype_score = pd.DataFrame({
    'Phenotype': result_phenotype,
    'Network_score': result_phenotype_score,
    'z-score': z_score_list,
    'P-value': P_value_list,
    'UMLSID': phenotype_umlsid
})

print(Phenotype_score)

Phenotype_score=Phenotype_score.sort_values(by='P-value',ascending=True)

#     # Compound ID에 해당하는 inferred phenotype.csv 생성
# if not os.path.exists('compound 결과'):
#     os.makedirs('compound 결과')
Phenotype_score.to_csv('result/{}차/{}_검증.csv'.format(number,number),index=False, encoding='UTF-8')
    
end = time.time()
sec = (end - start)
result = datetime.timedelta(seconds=sec)
print("걸린 시간 : ",result)
print(f"걸린 시간 : {end - start:.5f} sec")

# 엑셀파일이 'phenotype mappingv3.xlsx', '1_검증.csv' 이렇게 2개가 있어
#  'phenotype mappingv3.xlsx' 파일은 UMLSID, Group, GroupID 이렇게 3개의 컬럼이 존재하고
# '1_검증.csv' 파일은 Phenotype, Network_score, z-score, P-value, UMLSID 이렇게 5개의 컬럼이 존재해
# 여기서 '1_검증.csv' 파일 내에 있는 UMLSID의 데이터와  'phenotype mappingv3.xlsx' 파일 내에 있는 UMLSID의 데이터가 일치하면 Group, GroupID 값을 가져와서 똑같이 써주고, 아니면 None 값을 써주고 싶어
# 그래서 최종적으로 '1_검증.csv' 파일내에 Phenotype, Network_score, z-score, P-value, UMLSID, Group, GroupID 이렇게 7개의 컬럼이 존재하게 만들고 싶어
# 이렇게 코드 짜줘


# 'phenotype mappingv3.xlsx' 파일을 읽어옵니다.
mapping_df = pd.read_excel('phenotype mappingv3.xlsx', engine='openpyxl')

# '1_검증.csv' 파일 내의 UMLSID와 'phenotype mappingv3.xlsx' 파일 내의 UMLSID를 기반으로 병합합니다.
merged_df = Phenotype_score.merge(mapping_df, on='UMLSID', how='left')

# Group 및 GroupID가 없는 경우 None으로 채웁니다.
merged_df['Group'].fillna('None', inplace=True)
merged_df['GroupID'].fillna('None', inplace=True)

# 'Network_score' 컬럼의 데이터를 추출합니다.
network_scores = merged_df['Network_score']

# Min-Max Scaling을 0부터 1로 설정하여 적용합니다.
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_scores = scaler.fit_transform(network_scores.values.reshape(-1, 1))  # 2D 배열로 변환 후 스케일링

# 스케일링된 값을 'scaling_score' 컬럼에 추가합니다.
merged_df['scaling_score'] = scaled_scores

# 'P-value'의 값을 조정합니다.
def adjust_pvalue(value):
    if value == 999 or value == 1000:
        return 1
    else:
        return (value+1) / 1000

merged_df['P-value'] = merged_df['P-value'].apply(adjust_pvalue)

cols_to_drop = [col for col in merged_df.columns if col.startswith("Unnamed")]
merged_df = merged_df.drop(columns=cols_to_drop)

hpo_df=pd.read_csv("Data/[original] HPO phenotype data.csv")

merged_df=pd.merge(merged_df, hpo_df, left_on='Phenotype', right_on='phenotype name', how='left')

final_df=merged_df[['Phenotype','Network_score','z-score','P-value','UMLSID','Group','GroupID','scaling_score', 'hpo_id']]

# 수정된 DataFrame을 새로운 엑셀 파일로 저장합니다.
final_df.to_csv('result/{}차/{}_검증_with_scaling.csv'.format(number,number), index=False)  # 저장할 파일명을 지정해주세요.

phe Abnormality of metabolism/homeostasis
z 67.4414840407614
p 91
phe Limb dystonia
z 78.596903490746
p 82
phe Abnormality of prenatal development or birth
z 44.794603310318415
p 285
phe Cerebral dysmyelination
z 91.15845624581841
p 330
phe Trichorrhexis nodosa
z 736.1416603663372
p 10
phe Poor head control
z 39.159620019111024
p 0
phe Head titubation
z 220.44940786822482
p 1
phe Spastic diplegia
z 49.97047431016292
p 52
phe Low-set nipples
z 1174.2318457955646
p 2
phe Infra-orbital crease
z 981.0428221745076
p 0
phe Congenital exfoliative erythroderma
z 1656.1186790642655
p 2
phe Lower limb muscle weakness
z 78.99662313005948
p 0
phe Joint dislocation
z 50.9947698527705
p 0
phe Craniosynostosis
z 20.30659772351491
p 0
phe Prematurely aged appearance
z 111.9039407577389
p 7
phe Absence of subcutaneous fat
z 114.33591432717769
p 3
phe Partial agenesis of the corpus callosum
z 191.5007493350698
p 0
phe Cheyne-Stokes respiration
z 2333.3523805405734
p 92
phe Craniofacial dystonia
z 725.77

phe Progressive muscle weakness
z 27.6142102604179
p 0
phe Type 1 muscle fiber atrophy
z 38.66613725914405
p 81
phe Abnormality of the respiratory system
z 0.5723301050513904
p 904
phe Neuromuscular dysphagia
z 7.843188610478209
p 213
phe Gowers sign
z 70.91429652563971
p 28
phe Joint contracture of the hand
z 23.30720219409263
p 0
phe EMG: myopathic abnormalities
z 15.897940033372233
p 0
phe EMG: neuropathic changes
z 36.07846317737791
p 4
phe Abnormality of skeletal morphology
z 79.63535779485042
p 6
phe Muscle fiber splitting
z 139.69782848309643
p 23
phe Difficulty climbing stairs
z 36.78894360425019
p 6
phe Shoulder girdle muscle weakness
z 59.13080407048192
p 4
phe Intercostal muscle weakness
z 248.63028326503164
p 208
phe Respiratory failure requiring assisted ventilation
z 149.101162504418
p 170
phe Aspiration pneumonia
z 37.9916622876174
p 9
phe Scapular winging
z 9.762344271539652
p 0
phe Centrally nucleated skeletal muscle fibers
z 19.36853488444922
p 0
phe Generalized limb 

phe Decreased circulating IgG2 level
z 14.714983488982206
p 174
phe Thyroiditis
z 4.598257173609634
p 119
phe Recurrent fever
z 0.9303684526628805
p 0
phe Abnormal lymphocyte morphology
z 63.761749523627714
p 114
phe Severe combined immunodeficiency
z 40.76963145251851
p 0
phe Recurrent viral infections
z 5.027849903973302
p 0
phe Anterior rib cupping
z 66.79201096929637
p 9
phe Increased circulating IgE level
z 1.8674559257810834
p 93
phe Absent tonsils
z 614.7048809935974
p 3
phe Abnormality of the metaphysis
z 0.06509273710052485
p 21
phe Decreased circulating total IgM
z 5.712042663204996
p 4
phe Absence of lymph node germinal center
z 98.35174617875718
p 135
phe Recurrent fungal infections
z 6.801704730653233
p 0
phe Allergy
z 67.18156757867895
p 156
phe Recurrent pneumonia
z -0.8997013367329951
p 0
phe Aplasia of the thymus
z 310.46202804137516
p 62
phe Lack of T cell function
z 146.02083903197033
p 3
phe Abnormality of humoral immunity
z 29.93663566797581
p 106
phe Severe B lymp

phe Abnormal circulating methionine concentration
z 195.71599696678754
p 178
phe Reduced factor VII activity
z 84.32305753842513
p 71
phe Hyperhomocystinemia
z 54.778755520929515
p 8
phe Abnormality of hair texture
z 41.89498202301842
p 44
phe Elevated coagulation factor V activity
z 194.99769348942348
p 178
phe Reduced antithrombin III activity
z 89.66613910483127
p 0
phe Abnormality of coagulation
z -1.4498944909840048
p 175
phe Widened subarachnoid space
z 34.112539999765765
p 0
phe Hydrops fetalis
z 5.910339779426393
p 2
phe Alopecia universalis
z 12.603823672928792
p 50
phe Short corpus callosum
z 133.85678217436185
p 0
phe Aplasia/Hypoplasia of the thymus
z 55.89765243162298
p 9
phe Skin ulcer
z -2.2911042308180156
p 28
phe Congenital agranulocytosis
z 150.58540470375235
p 8
phe Decreased circulating antibody level
z -1.7006398060605712
p 82
phe Hypoplasia of the thymus
z 16.638628696904963
p 2
phe Abnormality of mitochondrial metabolism
z 2.6563816137054466
p 262
phe Impaired T 

phe Short ribs
z 2.4807818947553786
p 0
phe Phosphoethanolaminuria
z 312.19824967111583
p 0
phe Low alkaline phosphatase
z 46.02354663297345
p 0
phe Short lower limbs
z 18.524854882437666
p 3
phe Decreased calvarial ossification
z 6.074872127605358
p 30
phe Postaxial hand polydactyly
z -0.9776362535986459
p 49
phe Congenital conductive hearing impairment
z 2707.619578284989
p 289
phe Frontal cutaneous lipoma
z 2703.1837386186626
p 289
phe Median cleft lip
z 1.0938986314291468
p 19
phe Basal encephalocele
z 2682.36397941117
p 289
phe Bifid nose
z 35.9245353121924
p 7
phe Median cleft palate
z 32.92905983087272
p 77
phe Anterior basal encephalocele
z 2658.498510799406
p 289
phe Lipoma of corpus callosum
z 937.4329338183321
p 575
phe Cranium bifidum occultum
z 935.5636422330292
p 575
phe Bifid tongue
z 4.42684340478009
p 156
phe Finger clinodactyly
z 4.378851475879951
p 304
phe Hypopituitarism
z 6.380665300100209
p 58
phe Widely-spaced maxillary central incisors
z 14.230996246717751
p 991

...

phe Unilateral microphthalmos
z -1.2741512322341033
p 476
phe Abnormal cerebral cortex morphology
z -3.0257419564383836
p 48
phe Abnormal common carotid artery morphology
z -0.8030866969945549
p 866
phe Progressive ventriculomegaly
z -0.8089788887970034
p 866
phe Hyperextensible hand joints
z -1.9274996259786594
p 928
phe Secretory IgA deficiency
z -0.8184032804067896
p 866
phe Unilateral lung agenesis
z -1.6850770912793323
p 866
phe Bilateral renal dysplasia
z -0.9928814189945167
p 921
phe Prominent corneal nerve fibers
z -0.5012739592274374
p 256
phe Poorly ossified cervical vertebrae
z -0.625809883335022
p 333
phe Anterior tibial bowing
z -1.255427983231119
p 921
phe Shortening of all phalanges of the toes
z 0.36692987167409846
p 333
phe Dilated vestibule of the inner ear
z 1.2351806875792053
p 74
phe Hypoplasia of the semicircular canal
z -1.816068236652475
p 189
phe Dysmyelinating leukodystrophy
z -1.4090563747964313
p 44
phe Respiratory insufficiency due to defective ciliary clea

phe Triangular epiphysis of the proximal phalanx of the 3rd finger
z 1.0006779464488078
p 546
phe Abnormal coronary artery origin
z -0.609612610321482
p 366
phe Cyst of the ductus choledochus
z -3.3023177698430484
p 280
phe Hyperextensible thumb
z -2.2568552809595688
p 294
phe Curved fingers
z -2.2575657658369024
p 294
phe Abnormal duodenum morphology
z -1.8434375743955285
p 679
phe Ciliary body coloboma
z -2.3280629832471083
p 598
phe Muscular edema
z 0.26866343939953147
p 467
phe Brachial plexus neuropathy
z -1.8741919996324277
p 334
phe Limited knee flexion/extension
z -0.610656915126262
p 0
phe Absent muscle fiber dysferlin
z 0.24428558190295135
p 467
phe Hypolipidemia
z 36.07830661071411
p 147
phe Meconium peritonitis
z -0.9912256741928304
p 978
phe Reduced prothrombin consumption
z -0.8851537966478218
p 807
phe Absence of alpha granules
z -0.8852255055198583
p 807
phe Glue ear
z -1.5367908923988534
p 562
phe Monocular strabismus
z 3.247127042963106
p 0
phe Absent central microtub

phe Tarsal stippling
z -1.4364587603827805
p 411
phe Elevated 8(9)-cholestenol
z -1.705353755104705
p 325
phe Elevated 8-dehydrocholesterol
z -1.7057236005313015
p 325
phe Abnormal nasal bridge morphology
z -1.437720215827615
p 411
phe Renal calcium wasting
z -0.824740618519666
p 778
phe Hypocitraturia
z 39.22810977849943
p 127
phe Hypercitraturia
z 47.72303970813204
p 279
phe Bullet-shaped distal phalanx of the hallux
z -1.1351834037982618
p 474
phe Aplasia of the nose
z -2.808181977854117
p 381
phe Absent stapes
z -1.7789126186972226
p 669
phe Apical muscular ventricular septal defect
z -1.7796731491273856
p 669
phe Prominent palatine ridges
z -3.1282019205466436
p 322
phe Premature posterior fontanelle closure
z 0.6948135613915906
p 451
phe Brittle scalp hair
z 1.0034785994657263
p 271
phe Slow-growing nails
z 1.47426935370256
p 256
phe Abnormal circulating histidine concentration
z -0.541166402596777
p 144
phe Positive ferric chloride test
z 5.23674590334031
p 114
phe Abnormality o

phe Neoplasm of the urethra
z -1.2084998783710426
p 33
phe Enlarged flash visual evoked potentials
z -1.3017144127529854
p 269
phe Hypoplasia of the retina
z -1.304799325446957
p 269
phe Hand monodactyly
z -2.253500413239065
p 490
phe Triangular tongue
z -1.0428671382340071
p 385
phe Trapezoidal shaped vertebral bodies
z 50.43204784906376
p 64
phe Exercise-induced lactic acidemia
z -1.9058815604853059
p 99
phe Stage 1 chronic kidney disease
z -1.100236300821028
p 855
phe Sclerosis of finger phalanx
z -1.2762311141520017
p 961
phe Fatigable weakness of speech muscles
z -1.2767158549374567
p 961
phe Central nervous system cyst
z -1.4322476340043522
p 312
phe Double outlet right ventricle with doubly committed ventricular septal defect and pulmonary stenosis
z 2.485603800058819
p 39
phe Generalized papillary lesions
z -1.227016500408222
p 143
phe Internal carotid artery dissection
z 37.11170626292437
p 63
phe Congenital foot contractures
z -2.193407178788663
p 824
phe Left atrial isomeris

phe Unilateral polymicrogyria
z -1.475057267344319
p 41
phe Frontoparietal cortical dysplasia
z -1.475057267344319
p 41
phe Alveolar process hypoplasia
z -1.860212609024146
p 712
phe Pulmonary valve defects
z -1.8617394127785205
p 712
phe Unilateral conductive hearing impairment
z -2.0595043751627364
p 992
phe Abnormal renal cortex morphology
z -2.066428248658895
p 992
phe Mongolian blue spot
z 4.691313638073465
p 435
phe Anterior radial head dislocation
z -1.56365764909613
p 573
phe Difficulty adjusting to changes in luminance
z -1.8561643929261302
p 589
phe Excessive bleeding after a venipuncture
z -2.0586309509054606
p 989
phe Cervicomedullary schisis
z -0.5854921346418631
p 534
phe Multifocal atrial tachycardia
z -1.990409801065421
p 991
phe Symphalangism affecting the phalanges of the hallux
z -2.3588084028541347
p 470
phe Broad femoral head
z -2.4894206570665047
p 918
phe Cleft vertebral arch
z -2.4894206570665047
p 918
phe Bifid first metacarpal
z -2.516316485028674
p 918
phe Ab